This file imports Revelio sample data, performs data analysis, and saves plotly plots. The file structure is: 
/path/
│
├── data/
|   ├── Revelio data folders]
├── src/
|   ├── analysis.ipynb [this file]
|   ├── get_bls.py [utility to get state unem data]
|   ├── assignment_text.md [post text]
└── output/
    ├── [figures, html output]

In [46]:
# ---------------------------- Imports and helper functions --------------------
import os
current_dir = os.getcwd()
project_root = os.path.dirname(current_dir) if current_dir.endswith("src") else current_dir
os.chdir(project_root)

import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)
warnings.simplefilter("ignore", category=FutureWarning)

import numpy as np 
import pandas as pd 
import markdown 
import regex as re
import statsmodels.api as sm
import requests
from bs4 import BeautifulSoup as bs
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = 'vscode+notebook'
 
from src.get_bls import get_ur 

# list directories in /data/
data_list = os.listdir('data/')
files = {directory: os.listdir(os.path.join('data', directory)) for directory in data_list 
         if os.path.isdir(os.path.join('data', directory))}

# bring in data function 
def data_read(files, directory, filename=False):
    df_list = []
    for file in files[directory]:
        df = pd.read_csv(os.path.join('data', directory, file))
        if filename: 
            df['filename'] = file
        df_list.append(df)
    
    result_df = pd.concat(df_list, ignore_index=True)
    return result_df

In [2]:
# ------------------------------------- Data Import --------------------------------------
# Postings data import and basic variables
# bring in data 
post = data_read(files, 'postings_linkedin_individual')
post['post_date'] = pd.to_datetime(post['post_date'], format='%Y-%m-%d') 
post['remove_date'] = pd.to_datetime(post['remove_date'], format='%Y-%m-%d')
post['post_mo'] = post['post_date'].dt.to_period('M')
post['remove_mo'] = post['remove_date'].dt.to_period('M')
post['post_q'] = post['post_date'].dt.to_period('Q')
post['year'] = post['post_date'].dt.year
post.rename(columns={'ultimate_parent_company_name': 'upco'}, inplace=True) #for brevity  

# restrict to US and 2022+, identifiable state 
post = post[(post['post_mo']>='2022-01') & (post['country']=='United States') & (post['state']!='empty')]

#  duration of job ad and indicator for sampling frame 
post['dur'] = (post['remove_date'] - post['post_date']).dt.days
final_day = np.max(post['post_date'], axis=0)
durs = [15,30,45]
for d in durs:
    post[f'dur_{d}'] = (post['dur']>=d)| (post['remove_date'].isna()==True)
    post[f'dur_{d}_samp'] = (final_day-post['post_date']).dt.days>d

# salary terciles 
post['salary_tercile'] = pd.qcut(post['salary'], q=3, labels=[1,2,3])

In [3]:
# -------------------------- State unemployment data from BLS ---------------------------------
ur = get_ur('2022-01-01', '2024-10-31')
ur.loc[ur['state']=='District of Columbia', 'state'] = 'Washington, D.C.'
ur['year'] = ur['post_mo'].dt.year
ur_year = ur.groupby(['year','state'])['ur'].mean().reset_index()

In [4]:
# -------------------------- Workforce dynamics by geography ---------------------------------
# bring in wf geo data 
wfg = data_read(files, 'wf_dynam_geo_breakdown')
wfg['month'] = pd.to_datetime(wfg['month'], format='%Y-%m').dt.to_period('M')
wfg.describe(include='all')

wfg = wfg[(wfg['country']=='United States') & (wfg['month'] >= '2022-01')] 
wfg['cnt_wfg'] = 1
wfg['year'] = wfg['month'].dt.year

# collapse by month, state
wfg_state = (wfg
            .groupby(['month','state'])
            [['external_inflow','external_outflow','count']]
            .sum()
            .reset_index())
wfg_state.rename(columns={'month': 'post_mo', 'count': 'cnt_wfg'}, inplace=True)
wfg_state['year'] = wfg_state['post_mo'].dt.year
wfg_state['turn'] = 0.5 * (wfg_state['external_inflow'] + wfg_state['external_outflow'])/wfg_state['cnt_wfg']

# collapse by month, job
wfg_job = (wfg
            .groupby(['month','job_category'])
            [['external_inflow','external_outflow','count']]
            .sum()
            .reset_index())
wfg_job.rename(columns={'month': 'post_mo', 'count': 'cnt_wfg'}, inplace=True)
wfg_job['year'] = wfg_job['post_mo'].dt.year
wfg_job['turn'] = 0.5 * (wfg_job['external_inflow'] + wfg_job['external_outflow'])/wfg_job['cnt_wfg']

# collapse by month, state, job category 
wfg_state_job = (wfg
                 .groupby(['month','state','job_category'])
                 [['external_inflow','external_outflow','count']]
                 .sum()
                 .reset_index())
wfg_state_job.rename(columns={'month': 'post_mo', 'count': 'cnt_wfg'}, inplace=True)
wfg_state_job['year'] = wfg_state_job['post_mo'].dt.year
wfg_state_job['turn'] = 0.5 * (wfg_state_job['external_inflow'] + wfg_state_job['external_outflow'])/wfg_state_job['cnt_wfg']

In [5]:
# ----------------- Workforce dynamics by role, join to postings ------------------------
# (to provide prestige, seniority estimates)

# bring in wf role data 
wfr = data_read(files, 'wf_dynam_role_breakdown')
wfr['month'] = pd.to_datetime(wfr['month'], format='%Y-%m').dt.to_period('M')

# Keep to North America and sample period 
wfr = wfr[(wfr['region']=='Northern America') & (wfr['month'].dt.year >= 2022)]

# to properly weight seniority 
wfr['seniority_numerator'] = wfr['seniority']*wfr['count']

# collapse by company, role 
wfr_k150 = (wfr.groupby(['company','role_k150'])
            ['prestige_weight','total_prestige','seniority_numerator','count']
            .sum()
            .reset_index()
            .rename(columns={'company': 'upco', 'count': 'cnt_wfr'}))

# calculate weighted averages 
wfr_k150['prestige'] = wfr_k150['total_prestige']/wfr_k150['prestige_weight']
wfr_k150['seniority'] = wfr_k150['seniority_numerator']/wfr_k150['cnt_wfr']

# calculate terciles
for var in ['seniority', 'prestige']:
    wfr_k150.sort_values(var, inplace=True)
    wfr_k150['cum_cnt']   = wfr_k150['cnt_wfr'].cumsum()  
    wfr_k150['cum_share'] = wfr_k150['cum_cnt'] / wfr_k150['cum_cnt'].max()
    wfr_k150[f'{var}_tercile'] = np.ceil(wfr_k150['cum_share']*3)

wfr_k150.drop(['prestige_weight','seniority_numerator', 'total_prestige', 
               'cum_cnt','cum_share'], axis=1, inplace=True) 

wfr_k150.loc[wfr_k150['upco']=='Chewy, Inc.', 'upco'] = 'Argos Holdings, Inc.'

# merge on postings -- note that a small % do not match due to missing roles in postings
post = pd.merge(post, wfr_k150, on=['upco','role_k150'], how='left') 
print(len(post[post['cnt_wfr'].isna()==True]), "of", len(post), "unmatched")

In [6]:
# ------------- Aggregate postings data and join to other data ---------------------
aggdict1 = {f'dur_{d}': 'mean' for d in durs}
aggdict2 = {f'dur_{d}_samp': 'mean' for d in durs}
aggdict = aggdict1.copy()
aggdict.update(aggdict2)
aggdict['cnt'] = 'count'
post['cnt']=1

# job only (no ur)
post_job = (post.groupby(['post_mo','job_category'])
              .agg(aggdict)
              .reset_index())
post_job = pd.merge(post_job, wfg_job, on=['post_mo','job_category'], how='left')

# state only 
post_state = (post.groupby(['post_mo','state'])
              .agg(aggdict)
              .reset_index())
post_state = pd.merge(post_state, ur, on=['post_mo','state'], how='left')
post_state = pd.merge(post_state, wfg_state, on=['post_mo','state'], how='left')

# state and job 
post_state_job = (post.groupby(['post_mo','state','job_category'])
                  .agg(aggdict)
                  .reset_index())
post_state_job = pd.merge(post_state_job, ur, on=['post_mo','state'], how='left')
post_state_job = pd.merge(post_state_job, wfg_state_job, on=['post_mo','state','job_category'], how='left')

In [7]:
# ------ Aggregate to percentiles of state-month unem and state-job-month turnover ----- 

# set duration to use 
d = 30

# percentiles of state-month unem 
post_state.sort_values('ur', inplace=True)
post_state['cum_cnt'] = post_state['cnt'].cumsum()  
post_state['cum_share'] = post_state['cum_cnt'] / post_state['cum_cnt'].max()
post_state['ur_pctile'] = np.ceil(post_state['cum_share']*100)

post_state_pct = (post_state[post_state[f'dur_{d}_samp']==True]
                     .groupby(['ur_pctile'])[f'dur_{d}']
                     .mean().reset_index())

# percentiles of state-job-month turnover 
post_state_job.sort_values('turn', inplace=True)
post_state_job['cum_cnt'] = post_state_job['cnt'].cumsum()  
post_state_job['cum_share'] = post_state_job['cum_cnt'] / post_state_job['cum_cnt'].max()
post_state_job['turn_pctile'] = np.ceil(post_state_job['cum_share']*100)

post_state_job_pct = (post_state_job[post_state_job[f'dur_{d}_samp']==True]
                     .groupby(['turn_pctile'])[f'dur_{d}']
                     .mean().reset_index())

In [8]:
# ---------------  Time series aggregation at quarter-attribute level ------------
d = 30 
vars = ['salary', 'seniority', 'prestige']
post_qtr = (post[(post[f'dur_{d}_samp']==True) & (post['cnt_wfr'].isna()==False)]
            .groupby(['post_q'])[f'dur_{d}']
            .mean()
            .reset_index())

for var in vars:
    post_qtr_tmp = (post[(post[f'dur_{d}_samp']==True) & (post['cnt_wfr'].isna()==False)]
                    .groupby(['post_q', f'{var}_tercile'])[f'dur_{d}']
                    .mean()
                    .reset_index())
    post_qtr_tmp[f'{var}_tercile'] = post_qtr_tmp[f'{var}_tercile'].astype(int)
    post_qtr_tmp.rename(columns={f'dur_{d}': f'dur_{d}_{var}'}, inplace=True)
    post_qtr_tmp_wide = post_qtr_tmp.pivot(index='post_q', columns=f'{var}_tercile', values=f'dur_{d}_{var}')
    post_qtr_tmp_wide.columns = [f'dur_{d}_{var}{col}' for col in post_qtr_tmp_wide.columns]
    post_qtr_tmp_wide.reset_index(inplace=True)
    post_qtr = pd.merge(post_qtr, post_qtr_tmp_wide, on='post_q', how='left')
    
post_qtr['post_q'] = post_qtr['post_q'].dt.to_timestamp()

In [9]:
# ------------------------ Time series aggregation at month level ------------------------
d = 30 
vars = ['salary', 'seniority', 'prestige']
post_month = (post[(post[f'dur_{d}_samp']==True) & (post['cnt_wfr'].isna()==False)]
           .groupby(['post_mo'])[f'dur_{d}']
           .mean()
           .reset_index())

post_month['post_mo'] = post_month['post_mo'].dt.to_timestamp()

In [10]:
# define colors for plots 
colors=["#4baded", "#e76a87","#5ed996"]

In [66]:
# Figure 1: Overall job duration over time 
fig1 = px.line(post_month, x='post_mo', y='dur_30',
              color_discrete_sequence=colors,
);
fig1.update_traces(
    hovertemplate='In %{x|%B, %Y}, %{y:.0%} of job postings lasted a month or longer.<extra></extra>',
    hoverlabel=dict(bgcolor='white')
);
fig1.update_layout(
    width=700,
    height=550,
    margin=dict(l=10, r=10, t=40, b=10),
    plot_bgcolor='rgba(0,0,0,0)',  # Set background to clear
    title={
        'text': f'<b><span style="color:{colors[0]}">Job postings</span> have stayed up longer since the hot job market of 2022</b><br><sup>Percent of job postings that remain online for 30 days or longer</sup>',
        'x': 0.05 
    },
    xaxis=dict(
        showgrid=True,
        gridcolor='lightgray',  # Set gridlines to gray
        title=None,  # Remove x-axis title
        range=[post_month['post_mo'].min() - pd.DateOffset(months=1), post_month['post_mo'].max()]  # Add space to the left
    ),
    yaxis=dict(
        tickformat=".0%",
        showgrid=True,
        gridcolor='lightgray',  # Set gridlines to gray
        title=None  # Remove y-axis title
    )
);
fig1.show()
pio.write_html(fig1, file='output/fig1.html', auto_open=False,config=dict(displayModeBar=False))

In [67]:
# Fig 2: Job duration by prestige tercile
fig2 = px.line(post_qtr, x='post_q', y=['dur_30_prestige1', 'dur_30_prestige2', 'dur_30_prestige3'], 
              color_discrete_sequence=colors
);
fig2.update_traces(
    selector=dict(name='dur_30_prestige1'),
    hovertemplate='In %{x|Q%q %Y}, %{y:.0%} of job postings with low prestige lasted a month or longer.<extra></extra>',
    hoverlabel=dict(bgcolor='white')
);
fig2.update_traces(
    selector=dict(name='dur_30_prestige2'),
    hovertemplate='In %{x|Q%q %Y}, %{y:.0%} of job postings with medium prestige lasted a month or longer.<extra></extra>',
    hoverlabel=dict(bgcolor='white')
);
fig2.update_traces(
    selector=dict(name='dur_30_prestige3'),
    hovertemplate='In %{x|Q%q %Y}, %{y:.0%} of job postings with high prestige lasted a month or longer.<extra></extra>',
    hoverlabel=dict(bgcolor='white')
);
fig2.update_layout(
    width=700,
    height=550,
    margin=dict(l=10, r=10, t=40, b=10),
    plot_bgcolor='rgba(0,0,0,0)',  # Set background to clear
    title={
        'text': f'<b>Posting durations changed most for low- and mid-prestige roles</b><br><sup>Percent of job postings that remain online for 30 days or longer by <span style="color:{colors[0]}">low</span>, <span style="color:{colors[1]}">middle</span>, and <span style="color:{colors[2]}">high</span> prestige</sup>',
        'x': 0.05 
    },
    xaxis=dict(
        showgrid=True,
        gridcolor='lightgray', 
        title=None, 
        range=[post_qtr['post_q'].min() - pd.DateOffset(months=1), post_qtr['post_q'].max()]  # Add space to the left
    ),
    yaxis=dict(
        tickformat=".0%",
        showgrid=True,
        gridcolor='lightgray', 
        title=None 
    ),
    showlegend=False  
);
fig2.show()
pio.write_html(fig2, file='output/fig2.html', auto_open=False,config=dict(displayModeBar=False))

In [68]:
#Fig 3: Scatter duration vs urate
# linear fit
X = sm.add_constant(post_state_pct['ur_pctile'])
model = sm.OLS(post_state_pct['dur_30'], X).fit()
post_state_pct['fit'] = model.predict(X)

fig3 = px.scatter(post_state_pct, x='ur_pctile', y='dur_30', trendline='ols', color_discrete_sequence=colors);
fig3.add_trace(go.Scatter(x=post_state_pct['ur_pctile'], y=post_state_pct['fit'], mode='lines', name='Fit', line=dict(color=colors[1]), hoverinfo='none'));

fig3.update_traces(
    selector=dict(mode='markers'),
    hovertemplate='In percentile %{x} of state monthly unemployment, %{y:.0%} of job postings last a month or longer.<extra></extra>',
    hoverlabel=dict(bgcolor='white')
);
fig3.update_layout(
    width=700,
    height=550,
    margin=dict(l=10, r=10, t=40, b=10),
    plot_bgcolor='rgba(0,0,0,0)',
    title={
        'text': f'<b>Higher unemployment correlates with longer job postings</b><br><sup>Percent of job postings that remain online for 30 days or longer by monthly state unemployment percentile</sup>',
        'x': 0.05
    },
    xaxis=dict(
        showgrid=True,
        gridcolor='lightgray',
        title='Percentiles of monthly state unemployment, 2022-2024' 
    ),
    yaxis=dict(
        tickformat=".0%",
        showgrid=True,
        gridcolor='lightgray',
        title=None 
    ),
    showlegend=False  
);
fig3.show()
pio.write_html(fig3, file='output/fig3.html', auto_open=False,config=dict(displayModeBar=False))


In [69]:
# Fig 4: Scatter duration vs turnover
# linear fit
X = sm.add_constant(post_state_job_pct['turn_pctile'])
model = sm.OLS(post_state_job_pct['dur_30'], X).fit()
post_state_job_pct['fit'] = model.predict(X)

fig4 = px.scatter(post_state_job_pct, x='turn_pctile', y='dur_30', trendline='ols', color_discrete_sequence=colors);
fig4.add_trace(go.Scatter(x=post_state_job_pct['turn_pctile'], y=post_state_job_pct['fit'], mode='lines', name='Fit', line=dict(color=colors[1]), hoverinfo='none'));

fig4.update_traces(
    selector=dict(mode='markers'),
    hovertemplate='In percentile %{x} of job turnover, %{y:.0%} of job postings last a month or longer.<extra></extra>',
    hoverlabel=dict(bgcolor='white')
);
fig4.update_layout(
    width=700,
    height=550,
    margin=dict(l=10, r=10, t=40, b=10),
    plot_bgcolor='rgba(0,0,0,0)',
    title={
        'text': f'<b>Lower job turnover correlates with longer job postings</b><br><sup>Percent of job postings that remain online for 30 days or longer by job turnover percentile</sup>',
        'x': 0.05
    },
    xaxis=dict(
        showgrid=True,
        gridcolor='lightgray',
        title='Percentiles of job turnover by month, role, and state, 2022-2024' 
    ),
    yaxis=dict(
        tickformat=".0%",
        showgrid=True,
        gridcolor='lightgray',
        title=None 
    ),
    showlegend=False  
);
fig4.show()
pio.write_html(fig4, file='output/fig4.html', auto_open=False,config=dict(displayModeBar=False))
